# Exploration of Memphis and Nashville Barbecue Venues

## By 
C. J. Nyabando

The purpose of this project is to explore barbecue (BBQ) options in Memphis vs Nashville, Tennessee. Memphis is known for having great BBQ. There is even an annual BBQ festival held in May that attracts teams from all over the world. Is there an opportunity to popularize BBQ in Nashville in a similar manner? Memphis in May has been a great benefit for Memphis and the state of Tennessee. Is there an opportunity to bring a similar event to Nashville in the Fall to maximize the benefits for the state? The assumption of this exploratory study is that if there are as many BBQ restaurants in downtown Nashville as in downtown Memphis, there is a potential for introducing a comparable BBQ festival/completion in Nashville.

In [17]:
import requests # library to handle requests
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


In [ ]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
bbq = pd.DataFrame()
address = [ 'Memphis, TN', 'Nashville, TN']
for city in address:
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(city)
    latitude = location.latitude
    longitude = location.longitude
    print(city, latitude, longitude)
    search_query = ['BBQ','Barbeque']
    radius = 5000   
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    #results
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
    # tranform venues into a dataframe
    dataframe = json_normalize(venues)
    bbq= bbq.append(dataframe, sort=False)
bbq = bbq.reset_index(drop=True)
bbq.head()

In [ ]:
bbq.shape

In [6]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in bbq.columns if col.startswith('location.')] + ['id']
bbq_filtered = pd.DataFrame(bbq, columns=filtered_columns)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
bbq_filtered['categories'] = bbq_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
bbq_filtered.columns = [column.split('.')[-1] for column in bbq_filtered.columns]

bbq_filtered.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Central BBQ,BBQ Joint,147 E Butler Ave,US,Memphis,United States,at St Martin St,1764,"[147 E Butler Ave (at St Martin St), Memphis, ...","[{'label': 'display', 'lat': 35.13383433711528...",35.133834,-90.057185,NaN,38103,TN,4f8de1f7e4b0e77df30fbe61
1,Leonards BBQ,BBQ Joint,103 Main Street,US,Memphis,United States,Adams,377,"[103 Main Street (Adams), Memphis, TN 38103, U...",NaN,35.147995,-90.055577,NaN,38103,TN,4b2fb832f964a5208eee24e3
2,Quincy's BBQ Food Truck,Food Truck,NaN,US,Memphis,United States,NaN,279,"[Memphis, TN 38103, United States]","[{'label': 'display', 'lat': 35.14651844205082...",35.146518,-90.051436,NaN,38103,TN,5012c16de4b0b186e4255f76
3,BBQ On Wheels,Food Truck,NaN,US,NaN,United States,NaN,989,"[Tennessee, United States]","[{'label': 'display', 'lat': 35.140178, 'lng':...",35.140178,-90.052696,NaN,NaN,Tennessee,4e70fbb72271c7e78c09a82f
4,Central BBQ Truck,Food Truck,NaN,US,Memphis,United States,NaN,355,"[Memphis, TN 38134, United States]","[{'label': 'display', 'lat': 35.14588965399427...",35.145890,-90.052389,NaN,38134,TN,5002fd24e4b0ded98f2c475b


In [7]:
bbq_filtered.shape

(60, 16)

In [8]:
#Count of restaurants by city and category
bbq_count = bbq_filtered.groupby(['city','categories']).id.count()
bbq_count

city       categories           
Memphis    BBQ Joint                18
           Bar                       1
           Food                      2
           Food Truck                4
           General Entertainment     1
           Other Event               1
           Other Nightlife           1
Nashville  BBQ Joint                23
           Food                      5
           Mongolian Restaurant      1
           Music Venue               1
Name: id, dtype: int64

In [10]:
#create Nashville BBQ joints map

#filter Nashville BBQ joints
nash_bbq = bbq_filtered.loc[(bbq_filtered['city']=='Nashville')&(bbq_filtered['categories']=='BBQ Joint')]

#Nashville geocoordinates
latitude = 36.1622296
longitude = -86.7743531

map_nashbbq = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(nash_bbq['lat'], nash_bbq['lng'], nash_bbq['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        #fill=True,
        ##fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nashbbq)  
    
map_nashbbq

In [9]:
#create memphis BBQ joints map

#filter Memphis BBQ joints
memphis_bbq = bbq_filtered.loc[(bbq_filtered['city']=='Memphis')&(bbq_filtered['categories']=='BBQ Joint')]

#Memphis coordinates
latitude = 35.1490215
longitude = -90.0516285

map_membbq = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(memphis_bbq['lat'], memphis_bbq['lng'], memphis_bbq['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        #fill=True,
        ##fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_membbq)  
    
map_membbq

In [13]:
#get venue IDs
ids = bbq_filtered[['id']]
ids.head()

,id
0,4f8de1f7e4b0e77df30fbe61
1,4b2fb832f964a5208eee24e3
2,5012c16de4b0b186e4255f76
3,4e70fbb72271c7e78c09a82f
4,5002fd24e4b0ded98f2c475b


In [ ]:
#get data for each venue including ratings
for venue_id in ids ['id']:
    print (venue_id)
    #search for data for each venue using venue ID 
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID, CLIENT_SECRET,VERSION)
    # get results in json
    results = requests.get(url).json()
    try:
       print(result['response']['venue']['rating'])
    except:
       print('This venue has not been rated yet.')

4f8de1f7e4b0e77df30fbe61
This venue has not been rated yet.
4b2fb832f964a5208eee24e3
This venue has not been rated yet.
5012c16de4b0b186e4255f76
This venue has not been rated yet.
4e70fbb72271c7e78c09a82f
This venue has not been rated yet.
5002fd24e4b0ded98f2c475b
This venue has not been rated yet.
4b058661f964a5208d5f22e3
This venue has not been rated yet.
4f3212c419833175d60ddced
This venue has not been rated yet.
522226e811d2c12929eba959
This venue has not been rated yet.
519523c8498ef41408de3cbe
This venue has not been rated yet.
4fafd72ee4b08885ffc5fdeb
This venue has not been rated yet.
518f8cea498e9c894fcaf298
This venue has not been rated yet.
4dd9661e1f6ee1468353314f
This venue has not been rated yet.
569697f4498eb4bddc68e112
This venue has not been rated yet.
5afcc836cd441c002ca206ef
This venue has not been rated yet.
4fb68df4e4b0467a1d9eb247
This venue has not been rated yet.
5cddb3adb8fd9d002c2dc998
This venue has not been rated yet.
4fa47abce4b09571de770099
This venue has 